In [1]:
!pip install category_encoders
!pip install sklearn
from pathlib import Path

import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

import category_encoders as ce

import sklearn

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV


In [2]:
DATA_DIR = Path(r"./orig_data")
MOD_DATA_DIR = Path(r"./mod_data")
df_train = pd.read_csv(MOD_DATA_DIR / "train.csv")
df_test = pd.read_csv(MOD_DATA_DIR / "test.csv")
#df_test = pd.read_csv(MOD_DATA_DIR / "test_eval.csv")
df_network = pd.read_csv(DATA_DIR / "network.csv")

In [3]:
df_train.shape

(1905635, 9)

In [4]:
df_test.shape

(1259229, 10)

In [5]:
df_network.shape

(113, 3)

In [6]:
df_train.columns

Index(['id', 'date', 'lineName', 'directionCode', 'trainNo', 'stopStation',
       'planArrival', 'delayTime', 'prev_dalayTime'],
      dtype='object')

In [7]:
df_train.head(10)

,id,date,lineName,directionCode,trainNo,stopStation,planArrival,delayTime,prev_dalayTime
0,0,20191204,A,1,AFGKvJ,Rlfq,06:11,0.0,0.0
1,1,20191204,A,1,AFGKvJ,coZB,06:13,0.0,0.0
2,2,20191204,A,1,AFGKvJ,LMww,06:16,0.0,0.0
3,3,20191204,A,1,AFGKvJ,VNyR,06:18,0.0,0.0
4,4,20191204,A,1,AFGKvJ,jhlV,06:20,0.0,0.0
5,5,20191204,A,1,AFGKvJ,efzW,06:22,0.0,0.0
6,6,20191204,A,1,AFGKvJ,PcxI,06:25,0.0,0.0
7,7,20191204,A,1,AFGKvJ,ejfb,06:27,0.0,0.0
8,8,20191204,A,1,AFGKvJ,RDLf,06:28,0.0,0.0
9,9,20191204,A,1,AFGKvJ,cRgf,06:31,0.0,0.0


In [8]:
# Eoncodeしたい列をリストで指定。もちろん複数指定可能。
#list_cols = ['date','lineName','trainNo','stopStation','delay_case']
list_cols = ['date','lineName','trainNo','stopStation']

ce_oe = ce.OrdinalEncoder(cols=list_cols,handle_unknown='impute')
df_train = ce_oe.fit_transform(df_train)
df_test = ce_oe.fit_transform(df_test)


/opt/conda/lib/python3.8/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [9]:
df_train.head(10)

,id,date,lineName,directionCode,trainNo,stopStation,planArrival,delayTime,prev_dalayTime
0,0,1,1,1,1,1,06:11,0.0,0.0
1,1,1,1,1,1,2,06:13,0.0,0.0
2,2,1,1,1,1,3,06:16,0.0,0.0
3,3,1,1,1,1,4,06:18,0.0,0.0
4,4,1,1,1,1,5,06:20,0.0,0.0
5,5,1,1,1,1,6,06:22,0.0,0.0
6,6,1,1,1,1,7,06:25,0.0,0.0
7,7,1,1,1,1,8,06:27,0.0,0.0
8,8,1,1,1,1,9,06:28,0.0,0.0
9,9,1,1,1,1,10,06:31,0.0,0.0


In [10]:
#drop_columns = ["date", "planArrival"]
drop_columns = ["planArrival"]

df_train = df_train.drop(drop_columns, axis = 1)
df_train = df_train.drop("id", axis = 1)
df_test = df_test.drop(drop_columns, axis = 1)

#df_train['date'] = pd.to_datetime(df_train['date'])
#df_test['date'] = pd.to_datetime(df_test['date'])


In [11]:
df_train.head(10)

,date,lineName,directionCode,trainNo,stopStation,delayTime,prev_dalayTime
0,1,1,1,1,1,0.0,0.0
1,1,1,1,1,2,0.0,0.0
2,1,1,1,1,3,0.0,0.0
3,1,1,1,1,4,0.0,0.0
4,1,1,1,1,5,0.0,0.0
5,1,1,1,1,6,0.0,0.0
6,1,1,1,1,7,0.0,0.0
7,1,1,1,1,8,0.0,0.0
8,1,1,1,1,9,0.0,0.0
9,1,1,1,1,10,0.0,0.0


In [12]:
X = df_train.drop('delayTime', axis=1)
y = df_train['delayTime']

X_train,X_test,y_train,y_test = train_test_split(X,y,train_size=0.8,shuffle=True)


In [13]:
#ランダムフォレストによる学習
model=RandomForestClassifier(random_state=0,n_estimators=8)

model.fit(X_train,y_train)


RandomForestClassifier(n_estimators=12, random_state=0)

In [14]:
#正答率を出力
print('train acc: %.3f' % model.score(X_train, y_train))
print('test acc: %.3f' % model.score(X_test, y_test))

train acc: 0.992
test acc: 0.880


In [15]:
y_test.head(10)

1833713    0.0
1823401    0.0
68132      0.0
541383     3.0
1249546    1.0
388231     6.0
402208     0.0
727473     7.0
1872703    0.0
80614      0.0
Name: delayTime, dtype: float64

In [16]:
#target が 0のものを削除
drop_index = df_test.index[df_test['target'] == 0]
#条件にマッチしたIndexを削除
df_test = df_test.drop(drop_index)

df_test = df_test.drop('target', axis=1)
df_test = df_test.drop('delayTime', axis=1)
X_eval = df_test.drop("id", axis = 1)

X_eval.head(10)


,date,lineName,directionCode,trainNo,stopStation,prev_dalayTime
50,84,1,1,16.0,8,3.0
51,84,1,1,16.0,9,0.0
52,84,1,1,16.0,10,0.0
53,84,1,1,16.0,11,0.0
54,84,1,1,16.0,12,0.0
55,84,1,1,16.0,13,0.0
56,84,1,1,16.0,14,0.0
57,84,1,1,16.0,15,0.0
58,84,1,1,16.0,16,0.0
59,84,1,1,16.0,17,0.0


In [17]:
df_test.to_csv('test_eval_sample.csv')

In [18]:
result = model.predict(X_eval)

In [19]:
with open('submit_random_forest.csv', 'w') as f:
    for i in range(len(result)):
        write_string=str(int(df_test.iloc[i]['id'])) + ',' + str(result[i])
        f.write(write_string+'\n')

In [20]:
param_grid = {'n_estimators': [100], 
              'max_features': [1,'auto',None],
              'min_samples_leaf': [3,4,5,6],
              'n_jobs':[-1],
              'max_depth':[12]}


rfc = RandomForestClassifier(random_state=0)
#グリッドサーチで探索
model = GridSearchCV(estimator=rfc, scoring="accuracy", param_grid=param_grid, cv=5)

model.fit(X_train,y_train)

#正答率を出力
print('train acc: %.3f' % model.score(X_train, y_train))
print('test acc: %.3f' % model.score(X_test, y_test))


/opt/conda/lib/python3.8/site-packages/sklearn/model_selection/_split.py:670: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"


train acc: 0.894
test acc: 0.892


In [21]:
result = model.predict(X_eval)

with open('submit_grid_search.csv', 'w') as f:
    for i in range(len(result)):
        write_string=str(int(df_test.iloc[i]['id'])) + ',' + str(result[i])
        f.write(write_string+'\n')


In [22]:
!pip install xgboost
import xgboost as xgb

#model = xgb.XGBRegressor(objective ='reg:squarederror')

#XGBR = xgb.XGBRegressor(objective ='reg:squarederror')

#params = {'max_depth': [6], 'learning_rate': [0.3]}
#model = GridSearchCV(XGBR, params, cv = 5, refit=True,  scoring='r2', verbose = 1, n_jobs=1)

XGBR = xgb.XGBRegressor(n_estimators=100, objective='reg:squarederror', gamma=0, min_child_weight=1, subsample=1, colsample_bytree=1)
# モデルを作成。xgb.trainにおけるparamの部分

params = {'max_depth': [12], 'learning_rate': [0.3]}
model = GridSearchCV(XGBR, params, cv = 5, refit=True,  scoring='r2', verbose = 1, n_jobs=1)

model.fit(X_train,y_train)

#model.fit(X_train,y_train) 

#正答率を出力
print('train acc: %.3f' % model.score(X_train, y_train))
print('test acc: %.3f' % model.score(X_test, y_test))


Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:  2.0min finished


train acc: 0.969
test acc: 0.970


In [23]:
result = model.predict(X_eval)

with open('submit_xgboost.csv', 'w') as f:
    for i in range(len(result)):
        write_string=str(int(df_test.iloc[i]['id'])) + ',' + str(round(result[i], 0))
        f.write(write_string+'\n')